# Moderated Chat application samples with `AmazonComprehendModerationChain`
---

Install the necessary libraries. We use [Gradio](https://www.gradio.app/) for the Chat UI interface.

<div class="alert alert-block alert-info"> <b>NOTE:</b> You must use a Python3 kernel to run this notebook.</div>

In [ ]:
%pip install -U gradio boto3 nltk --quiet

In [ ]:
%pip install -U langchain_experimental huggingface_hub --quiet

In [ ]:
%pip install -U langchain pydantic --quiet

## Sample moderated chatbot with Amazon Bedrock
---

You can use the sample prompt below to start testing the chatbot:

```
What is John Doe's address, phone number and SSN from the following text?

John Doe, a resident of 1234 Elm Street in Springfield, recently celebrated his birthday on January 1st. Turning 43 this year, John reflected on the years gone by. He often shares memories of his younger days with his close friends through calls on his phone, (555) 123-4567. Meanwhile, during a casual evening, he received an email at johndoe@example.com reminding him of an old acquaintance's reunion. As he navigated through some old documents, he stumbled upon a paper that listed his SSN as 123-45-6789, reminding him to store it in a safer place.
```

In [ ]:
import boto3
from getpass import getpass
import os

comprehend_client = boto3.client('comprehend', region_name='us-east-1')
bedrock = boto3.client('bedrock-runtime', region_name='us-east-1')

model_id = "<your-model-id>"
model_id="anthropic.claude-instant-v1"

Get your API Key from Huggingface hub - https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [ ]:
from langchain.llms import Bedrock
from langchain.prompts import PromptTemplate
from langchain_experimental.comprehend_moderation import (AmazonComprehendModerationChain,                                                          
                                                         BaseModerationConfig, 
                                                         ModerationPromptSafetyConfig, 
                                                         ModerationPiiConfig, 
                                                         ModerationToxicityConfig)
import gradio as gr
import os


config_filter = {"PII": ModerationPiiConfig(threshold=0.5, labels=["SSN", "CREDIT_DEBIT_NUMBER"], mask_character="X", redact=True), 
                 "Toxicity": None, 
                 "Prompt-safety": None}

def respond(message, chat_history):    
    template = "\n\nHuman:{question}\n\nAssistant:"

    prompt = PromptTemplate(template=template, input_variables=["question"])
    bedrock_llm = Bedrock(client=bedrock, model_id=model_id, model_kwargs={'temperature':0, "stop_sequences": ["\n\nHuman:","</answer>"]})
    
    filters = [value for value in config_filter.values() if value is not None]
    # define different moderation configs using the filter configs above
    mod_config = BaseModerationConfig(filters=filters)
    
    mod_config = BaseModerationConfig(filters=filters) if filters else BaseModerationConfig()
    comprehend_moderation = AmazonComprehendModerationChain(client=comprehend_client, 
                                                            moderation_config = mod_config,
                                                            verbose=False)

    try:
        chain = (prompt 
                 | comprehend_moderation 
                 | {"input": (lambda x: x['output'] ) | bedrock_llm}
                 | comprehend_moderation 
                )
        response = chain.invoke({"question": message})
        return response['output']
    except Exception as e:
        error_msg = f"<b style='color:red;'>Error: {str(e)}</b>"
        return error_msg

def on_select(evt: gr.SelectData):
    if evt.target.label == "PII":
        config_filter[evt.target.label] = None if not evt.value else ModerationPiiConfig(threshold=0.5, labels=["SSN", "CREDIT_DEBIT_NUMBER"], mask_character="X", redact=True)
    elif evt.target.label == "Toxicity":
        config_filter[evt.target.label] = None if not evt.value else ModerationToxicityConfig(threshold=0.6)
    elif evt.target.label == "Prompt-safety":
        config_filter[evt.target.label] = None if not evt.value else ModerationPromptSafetyConfig(threshold=0.8)
    return f"You selected {evt.value} at {evt.index} from {evt.target} and selection is {evt.selected}"

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.ChatInterface(respond)
        with gr.Column(scale=0):
            pii = gr.Checkbox(label="PII", value=True, interactive=True) 
            pii.select(on_select)
            tox = gr.Checkbox(label="Toxicity", value=False, interactive=True)                
            tox.select(on_select)
            intent = gr.Checkbox(label="Prompt-safety", value=False, interactive=True)                
            intent.select(on_select)

demo.launch()

## Sample moderated chatbot with Hugging Face Hub model
---

You can use the sample prompt below to start testing the chatbot:

```
What is John Doe's address, phone number and SSN from the following text?

John Doe, a resident of 1234 Elm Street in Springfield, recently celebrated his birthday on January 1st. Turning 43 this year, John reflected on the years gone by. He often shares memories of his younger days with his close friends through calls on his phone, (555) 123-4567. Meanwhile, during a casual evening, he received an email at johndoe@example.com reminding him of an old acquaintance's reunion. As he navigated through some old documents, he stumbled upon a paper that listed his SSN as 123-45-6789, reminding him to store it in a safer place.
```

In [ ]:
import boto3
from getpass import getpass
import os

comprehend_client = boto3.client('comprehend', region_name='us-east-1')

Get your API Key from Huggingface hub - https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
from langchain import HuggingFaceHub
from langchain import PromptTemplate, LLMChain
from langchain_experimental.comprehend_moderation import (AmazonComprehendModerationChain,                                                          
                                                         BaseModerationConfig, 
                                                         ModerationPromptSafetyConfig, 
                                                         ModerationPiiConfig, 
                                                         ModerationToxicityConfig)
import gradio as gr
import os


config_filter = {"PII": ModerationPiiConfig(threshold=0.5, labels=["SSN", "CREDIT_DEBIT_NUMBER"], mask_character="X", redact=True), 
                 "Toxicity": None, 
                 "Prompt-safety": None}

def respond(message, chat_history):    
    repo_id = "google/flan-t5-xxl" # See https://huggingface.co/models?pipeline_tag=text-generation&sort=downloads for some other options

    template = """Question: {question}"""

    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm = HuggingFaceHub(
        repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 256}
    )
    
    filters = [value for value in config_filter.values() if value is not None]
    # define different moderation configs using the filter configs above
    mod_config = BaseModerationConfig(filters=filters)
    
    mod_config = BaseModerationConfig(filters=filters) if filters else BaseModerationConfig()
    comprehend_moderation = AmazonComprehendModerationChain(client=comprehend_client, 
                                                            moderation_config = mod_config,
                                                            verbose=False)

    try:
        chain = (prompt 
                 | comprehend_moderation 
                 | {"input": (lambda x: x['output'] ) | llm}
                 | comprehend_moderation 
                )
        response = chain.invoke({"question": message})
        return response['output']
    except Exception as e:
        error_msg = f"<b style='color:red;'>Error: {str(e)}</b>"
        return error_msg

def on_select(evt: gr.SelectData):
    if evt.target.label == "PII":
        config_filter[evt.target.label] = None if not evt.value else ModerationPiiConfig(threshold=0.5, labels=["SSN", "CREDIT_DEBIT_NUMBER"], mask_character="X", redact=True)
    elif evt.target.label == "Toxicity":
        config_filter[evt.target.label] = None if not evt.value else ModerationToxicityConfig(threshold=0.6)
    elif evt.target.label == "Prompt-safety":
        config_filter[evt.target.label] = None if not evt.value else ModerationPromptSafetyConfig(threshold=0.8)
    return f"You selected {evt.value} at {evt.index} from {evt.target} and selection is {evt.selected}"

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.ChatInterface(respond)
        with gr.Column(scale=0):
            pii = gr.Checkbox(label="PII", value=True, interactive=True) 
            pii.select(on_select)
            tox = gr.Checkbox(label="Toxicity", value=False, interactive=True)                
            tox.select(on_select)
            intent = gr.Checkbox(label="Prompt-safety", value=False, interactive=True)                
            intent.select(on_select)

demo.launch()

#### Troubleshooting
---

If you receive the following error

```
'Comprehend' object has no attribute 'detect_toxic_content'
```

this means that your Boto3 is out of date, if you have installed the updates already then please restart the notebook kernel using the controls above or by running the following piece of code.


In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)